In [1]:
import glob
import numpy as np
import re
import os

import IONEX
import datetime
import io
from pathlib import Path


os.path.abspath('.')

'l:\\PhD\\TEC\\utilities'

In [83]:

# file_path = os.path.join('..','ionex_c1p','2014','c1pg3650.14i')

file_path = os.path.join('..','ionex_c1p','2014','c1pg1150.14i')
file = open(file_path,'r')
data = file.read()
file.close()
file_path

'..\\ionex_c1p\\2014\\c1pg1150.14i'

In [84]:
def start_of_map_string(map_index,n_chars=80):

    result = "{}"+" "*54
    result = result.format(map_index+1)+"START OF RMS MAP    "
    result = " "*(n_chars-len(result))+result+"\n"

    return result

def end_of_map_string(map_index,n_chars=80):

    result = "{}"+" "*54
    result = result.format(map_index+1)+"END OF RMS MAP      "
    result = " "*(n_chars-len(result))+result+"\n"

    return result

def epoch_of_current_map_string(filename,map_index,n_chars=80):

    ionex_date = IONEX.ionex_filename_to_date(filename)

    hour_number = map_index*2

    hour = "{}".format(hour_number%24)
    hour = " "*(6-len(hour))+hour
    
    hour_padding = (" "*5+"0")
    hour_padding += hour_padding

    days_to_advance = int(hour_number//24)
    ionex_date += datetime.timedelta(days=days_to_advance)

    #https://stackoverflow.com/questions/904928/python-strftime-date-without-leading-0
    year  = ionex_date.strftime("  %Y")
    month = ionex_date.strftime("%#m")
    day   = ionex_date.strftime("%#d")

    month = " "*(6-len(month))+month
    day = " "*(6-len(day))+day

    text = "EPOCH OF CURRENT MAP\n"

    prefix = year+month+day+hour+hour_padding

    result = prefix+" "*(n_chars+1-len(prefix)-len(text))+text

    return result

def map_latitude_string(latitude_index):

    posfix = "-180.0 180.0   5.0 450.0                            LAT/LON1/LON2/DLON/H\n"
    lat = "{}".format(87.5 - latitude_index * 2.5)
    lat = " "*(8-len(lat))+lat
    result = lat+posfix
    return result


def map_matrix2string_ionex(matrix,inonex_map_columns=16):
    n=inonex_map_columns
    test_map_str = [["   "+'   '.join(map(str, arr[i:i + n])) for i in range(0, len(arr), n)] for arr in matrix]
    test_map_str = ['\n'.join(map(str, l)) for l in test_map_str]
    return test_map_str


def shrink_ionex_to_13_maps(ionex_lines_content):
    START_OF_MAP = "START OF TEC MAP"
    END_OF_MAP = "END OF TEC MAP"
    # test if shrinking required
    ionex_content = "".join(ionex_lines_content)
    n_maps = len(ionex_content.split(START_OF_MAP)[1:])
    if n_maps <= 13 : return ionex_lines_content

    header_to_replace = {'INTERVAL':'7200','# OF MAPS IN FILE':'13'}


    start_map_indecies_to_replace = {i+1:0 for i in range(n_maps-1,0,-1)}
    end_map_indecies_to_replace = {i+1:0 for i in range(n_maps-1,0,-1)}

    map_count=1
    end_map_count = 1



    for i,line in enumerate(ionex_lines_content):

        for key,val in header_to_replace.items():

            if key in line:
                # print(line)
                # print(re.sub('\d+', val, line))
                ionex_lines_content[i] = re.sub('\d+', val, line)

        if START_OF_MAP in line:
            # print(i,line)
            start_map_indecies_to_replace[map_count] = i
            map_count+=1

        if END_OF_MAP in line:
            # print(i,line)
            end_map_indecies_to_replace[end_map_count] = i
            end_map_count+=1

    #replace START OF TEC MAP indecies
    for key,index in start_map_indecies_to_replace.items():

        new_key = key//2+1
        new_index = start_map_indecies_to_replace[new_key]

        ionex_lines_content[index] = ionex_lines_content[new_index]

    for key,index in end_map_indecies_to_replace.items():

        new_key = key//2+1
        new_index = end_map_indecies_to_replace[new_key]

        ionex_lines_content[index] = ionex_lines_content[new_index]

    ionex_content = "".join(ionex_lines_content)

    split_start_maps = ionex_content.split(START_OF_MAP)
    header = split_start_maps[0]

    maps_string_list = split_start_maps[1:]
    map_indecies_to_select = list(range(0,26,2))
    shrinked_map_list = [maps_string_list[i] for i in map_indecies_to_select]

    shrinked_content = header+START_OF_MAP+START_OF_MAP.join(shrinked_map_list)

    list_content = [l+"\n" for l in shrinked_content.split('\n')]

    return list_content


def dmd_rms_ionex(c1p_file_path,dmd_predicted_maps,_replace='c1p',_replace_with='dmd_rms',debug=False):
    data = ''
    file_path = c1p_file_path
    maps = dmd_predicted_maps[0]

    for map_index,_map in enumerate(maps):

        data += start_of_map_string(map_index)
        data += epoch_of_current_map_string(c1p_file_path,map_index)

        maps_string = map_matrix2string_ionex(_map)
        for lat_index,map_array in enumerate(_map):

            data += map_latitude_string(lat_index)
            data += maps_string[lat_index]+'\n'
        
        data += end_of_map_string(map_index)

    with io.open(file_path,'r',newline='\n') as f:
        ionex_lines_content = f.readlines()


    content = shrink_ionex_to_13_maps(ionex_lines_content)



    # with io.open(file_path,'r',newline='\n') as f:
    #     lines_content = f.readlines()

    lines_content = content
    lines_content.insert(-1,data)
    lines_content = "".join(lines_content)

    dmd_file = file_path.replace(_replace,_replace_with)

    if debug : print(dmd_file,os.path.dirname(dmd_file))
    Path(os.path.dirname(dmd_file)).mkdir(parents=True, exist_ok=True)
    with io.open(dmd_file,'w',newline='\n') as f:
        f.write(lines_content)





# start_of_map = start_of_map_string(12)
# epoch_of_map = epoch_of_current_map_string(file_path,11)
# lat_lon_line = map_latitude_string(31)

# print(start_of_map,len(start_of_map))
# print(epoch_of_map,len(epoch_of_map))
# print(lat_lon_line,len(lat_lon_line))


pred_maps = np.random.randint(1,10,13*71*73).reshape((1,13,71,73))

dmd_rms_ionex(file_path,pred_maps)


In [39]:
test_map = np.zeros((4,32),dtype=int) #+ np.array(list(range(1,72))).reshape(-1,1)
n=16
test_map_str = [["   "+'   '.join(map(str, arr[i:i + n])) for i in range(0, len(arr), n)] for arr in test_map]

test_map_str = ['\n'.join(map(str, l)) for l in test_map_str]

# test_map_str = ["   "+'   '.join(map(str, arr)) for arr in test_map]

# arr =test_map[0,:]
# ' '.join(map(str, arr))
# test_map_str

# map_matrix2string_ionex(test_map)

pred_maps =  np.expand_dims(np.repeat(test_map[np.newaxis,:, :], 13, axis=0), axis=0)
pred_maps = np.random.randint(1,10,13*71*73).reshape((1,13,71,73))
# pred_maps = np.random.rand(1,13,71,73)
pred_maps.shape

(1, 13, 71, 73)

0


In [36]:
test_list = ['1','2','3','4','5','6','7','8','9','10']

n=3

output=[test_list[i:i + n] for i in range(0, len(test_list), n)]
print(output)

[['1', '2', '3'], ['4', '5', '6'], ['7', '8', '9'], ['10']]


In [37]:
data = ''
with open(file_path,'r') as f:
    line  = f.readline()
    map_count = 0
    lat_count = 0
    while line:
        data += line#+"\n"
        # if "START OF TEC MAP" in line:
        #     line  = f.readline()
        #     data += line+"\n"
        if "LAT/LON1/LON2/DLON/H" in line:
            try:
                for _ in range(5):
                    line  = f.readline()
                data += test_map_str[lat_count]+"\n"
                lat_count+=1
            except IndexError as ie:
                #print(map_count,lat_count,line)
                pass
        if "END OF TEC MAP" in line:
            # print(map_count,lat_count)
            map_count+=1
            lat_count=0

        line  = f.readline()
        if "START OF RMS MAP" in line:
            break
    while line:
        data += line#+"\n"
        line  = f.readline()
    # print(map_count)
# data



dmd_file = file_path.replace('c1p','dmd')

print(dmd_file,os.path.dirname(dmd_file))
Path(os.path.dirname(dmd_file)).mkdir(parents=True, exist_ok=True)
with open(dmd_file,'w') as f:
    f.write(data)

..\ionex_dmd\2013\dmdg1150.14i ..\ionex_dmd\2013


In [19]:

def map_matrix2string_ionex(matrix,inonex_map_columns=16):
    n=inonex_map_columns
    test_map_str = [["   "+'   '.join(map(str, arr[i:i + n])) for i in range(0, len(arr), n)] for arr in matrix]
    test_map_str = ['\n'.join(map(str, l)) for l in test_map_str]
    return test_map_str


def dmd_ionex(c1p_file_path,dmd_predicted_maps,_replace='c1p',_replace_with='dmd',debug=False):
    data = ''
    file_path = c1p_file_path
    maps = dmd_predicted_maps[0]
    with open(file_path,'r') as f:
        line  = f.readline()
        map_count = 0
        lat_count = 0
        test_map_str = map_matrix2string_ionex(maps[map_count])
        while line:
            
            data += line
            if "LAT/LON1/LON2/DLON/H" in line:
                try:
                    for _ in range(5):
                        line  = f.readline()
                    data += test_map_str[lat_count]+"\n"
                    lat_count+=1
                except IndexError as ie:
                    if debug: print('ERROR : ',map_count,lat_count,line)
                    pass
            if "END OF TEC MAP" in line:
                map_count+=1
                lat_count=0
                try:
                    test_map_str = map_matrix2string_ionex(maps[map_count])
                except:
                     if debug: print('ERROR : ',map_count,maps.shape)

            line  = f.readline()
            if "START OF RMS MAP" in line:
                break
        while line:
            data += line
            line  = f.readline()

    dmd_file = file_path.replace(_replace,_replace_with)

    if debug : print(dmd_file,os.path.dirname(dmd_file))
    Path(os.path.dirname(dmd_file)).mkdir(parents=True, exist_ok=True)
    with open(dmd_file,'w') as f:
        f.write(data)
    

dmd_ionex(file_path,pred_maps)


In [1]:
import time,threading,logging


def thread_function(name,b):
    logging.info("Thread %s: starting %s", name,b)
    time.sleep(b)
    logging.info("Thread %s: finishing %s", name,b)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    logging.info("Main    : before creating thread")

    for i in range(10):
        x = threading.Thread(target=thread_function, args=(i,i,))
        logging.info("Main    : before running thread")
        x.start()
    logging.info("Main    : wait for the thread to finish")
    # x.join()
    logging.info("Main    : all done")